Streaming data dan data preprocessing Kelompok 5 PDB

12S18002 : Wiranda Siahaan

12S18009 : Novita Enjelia Hutapea

12S18041 : Merika H. Manurung

12S18046 : Tiara Octavia Situmorang

# Consume twitter steaming data

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import numpy as np
import pandas as pd

In [ ]:
#Kindly put your credentials received from twitter developer account
consumer_key='2mTmxHwkZramjVDOewE2dybgQ'
consumer_secret='dJ42E93prTAze5BoF4wAIcxYA6gXfu9rGrHr6gguUt85xcHJSi'
access_token ='1517128047019040768-YMZCcFIf6VLbZXNkJsa79sZk70xFkM'
access_secret='Auwa9WPIktjUCrgQmzceZsdMtMA2hbnebSR7QlmYEdrlq'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [ ]:
for tweet in api.search(q='miniso', lan='en'):    
    print(tweet.text)

@convomfs 3 ini. Tp yang British rose jarang pake.
Yang miniso body mist sih. Gue bukan ahli parfum jadi semuanya s… https://t.co/slRCTCaRWj
@Pranaa__ awet gk si parfumnya miniso?
RT @FergieGo: Me encontré a la @JenySweet1 en el Miniso. https://t.co/f8KSXfhqw3
miniso has these and theyre so cute 😫 https://t.co/Wr0jqTA4yB
@convomfs Original Source Vanilla Raspberry, Zwitsal, Miniso Red Fruits &amp; Musk, Heaven Scent yg Black Amber sama Wo… https://t.co/T0Vd26tkry
Miniso turned into a gym and he was doing weights infront of me🥲
@tanyakanrl Beli sendok sama sumpit di miniso
@capriesed baru sembuh kemaren abis sakit huhuhu, wah parfum miniso emg lumayan enak2 ya wanginyaa
@OlixYT Yes but I was too shy too ask 😅 it's okay if no for me I appreciate it
Ir a miniso o el mejor parche del mundo?
eriask! Wta figure exo miniso sehun ridi ina harga nya brp ya?
˗ˏˋ 50฿ รวมส่ง กระเป๋าสะพายข้างเอลโม่ซื้อมาจาก miniso ´ˎ˗

#ส่งต่อกระเป๋า #ส่งต่อ #กระเป๋า #กระเป๋าสะพาย… https://t.co/1AfbCTbLOA
@miniso_p

In [ ]:
tweetDf = pd.DataFrame(columns = ['User', 'User_statuses_count', 
                             'user_followers','fav_count','User_location','Tweets',])

In [ ]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100000, lang='en').items():
        print(i, end='\r')
        tweetDf.loc[i, 'User'] = tweet.user.name
        tweetDf.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        tweetDf.loc[i, 'user_followers'] = tweet.user.followers_count
        tweetDf.loc[i, 'fav_count'] = tweet.favorite_count
        tweetDf.loc[i, 'User_location'] = tweet.user.location
        tweetDf.loc[i, 'Tweets'] = tweet.text
        
        
        tweetDf.to_csv('{}.csv'.format(file_name))
        i+=1
        if i == 200:
            break
        else:
            pass

In [ ]:
stream(data = ['miniso'], file_name = 'miniso_2021')

In [ ]:
tweetDf.head()

,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,mitski stans tehran⁷ ☼𓃬,23627,883,0,she/her/not my main account,miniso has these and theyre so cute 😫 https://...
1,lelee | kinda ia.,4837,72,0,ini rant berkedok main,"@convomfs Original Source Vanilla Raspberry, Z..."
2,Upadhya,15384,2775,5,Sri Lanka,Miniso turned into a gym and he was doing weig...
3,JHONRAFFY MORTIFERO,359,21,0,Philippines,@OlixYT Yes but I was too shy too ask 😅 it's o...
4,FCC ID,86152,1286,0,United States,MINISO Corporation has filed for FCC ID approv...


## Data Preprocessing

In [ ]:
# libraries
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.preprocessing import StandardScaler # used for feature scaling

In [ ]:
dataset = pd.read_csv('miniso_2021.csv') # to import the dataset into a variable
# Splitting the attributes into independent and dependent attributes
X = dataset.iloc[:, :-1].values # attributes to determine dependent variable / Class
Y = dataset.iloc[:, -1].values # dependent variable / Class

In [ ]:
dataset.head()

,Unnamed: 0,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,0,mitski stans tehran⁷ ☼𓃬,23627,883,0,she/her/not my main account,miniso has these and theyre so cute 😫 https://...
1,1,lelee | kinda ia.,4837,72,0,ini rant berkedok main,"@convomfs Original Source Vanilla Raspberry, Z..."
2,2,Upadhya,15384,2775,5,Sri Lanka,Miniso turned into a gym and he was doing weig...
3,3,JHONRAFFY MORTIFERO,359,21,0,Philippines,@OlixYT Yes but I was too shy too ask 😅 it's o...
4,4,FCC ID,86152,1286,0,United States,MINISO Corporation has filed for FCC ID approv...


In [ ]:
print(dataset.shape)

(200, 7)


### drop coloumn

In [ ]:
to_drop = ['Unnamed: 0', 'User', 'User_statuses_count', 'user_followers', 'fav_count', 'User_location']

In [ ]:
dataset.drop(to_drop, inplace=True, axis = 1)

In [ ]:
dataset.head()

,Tweets
0,miniso has these and theyre so cute 😫 https://...
1,"@convomfs Original Source Vanilla Raspberry, Z..."
2,Miniso turned into a gym and he was doing weig...
3,@OlixYT Yes but I was too shy too ask 😅 it's o...
4,MINISO Corporation has filed for FCC ID approv...


In [ ]:
print(dataset.shape)

(200, 1)


### missing value

In [ ]:
# handling the missing data and replace missing values with nan from numpy and replace with mean of all the other values
dataset1 = dataset.dropna()
print(dataset1.shape)

(200, 1)


# Text Preprocessing



In [ ]:
import re, string, unicodedata
import json
import nltk
import json
import time
import itertools  
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import tensorflow.keras.preprocessing.text as kpt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

## Case folding

In [ ]:
# Change text column to lower lower case
dataset1['Tweets'] = dataset1['Tweets'].str.lower()

# Look at the change
print(dataset1)

                                                Tweets
0    miniso has these and theyre so cute 😫 https://...
1    @convomfs original source vanilla raspberry, z...
2    miniso turned into a gym and he was doing weig...
3    @olixyt yes but i was too shy too ask 😅 it's o...
4    miniso corporation has filed for fcc id approv...
..                                                 ...
195  rt @sugarsanriio: hello kitty cooler bag from ...
196  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
197  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
198   @wana____ miniso taught me shege that’s why lool
199  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...

[200 rows x 1 columns]


## Remove URL

In [ ]:
def clean_data(dataset2):
#replace URL of a text
    dataset1['Tweets'] = dataset2['Tweets'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

clean_data(dataset1)
print(dataset1['Tweets']);

0                miniso has these and theyre so cute 😫  
1      @convomfs original source vanilla raspberry, z...
2      miniso turned into a gym and he was doing weig...
3      @olixyt yes but i was too shy too ask 😅 it's o...
4      miniso corporation has filed for fcc id approv...
                             ...                        
195    rt @sugarsanriio: hello kitty cooler bag from ...
196    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
197    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
198     @wana____ miniso taught me shege that’s why lool
199    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
Name: Tweets, Length: 200, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Number

In [ ]:
# removing numbers from strings of speciafied 
# column, here 'Name'
dataset1['Tweets']= dataset1['Tweets'].str.replace('\d+', '')
  
# display output with numbers removed from 
# required strings
print(dataset1)

                                                Tweets
0              miniso has these and theyre so cute 😫  
1    @convomfs original source vanilla raspberry, z...
2    miniso turned into a gym and he was doing weig...
3    @olixyt yes but i was too shy too ask 😅 it's o...
4    miniso corporation has filed for fcc id approv...
..                                                 ...
195  rt @sugarsanriio: hello kitty cooler bag from ...
196  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
197  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
198   @wana____ miniso taught me shege that’s why lool
199  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...

[200 rows x 1 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Punctuation

In [ ]:
dataset1['Tweets'] = dataset1['Tweets'].str.replace(r'[^\w\s]','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'\s*\d+\s*', '', text)
    text = re.sub('/[^\w\s]/gi', '', text)
    text = re.sub("@[^0-9a-zA-Z]+", '',text)
    text = re.sub(r'@[^\s]+', ' ', text)
    text = re.sub(r'#[^\w]+', ' ', text)
    text = re.sub(r"[.,:;+!\-_<^/=?&|$\"\(\)\d\*]", " ", text)
    text = re.sub('  +', ' ', text) 
    text = re.sub('\n+',' ',text)
    text = re.sub('\n', text)
    text = re.sub('\ ■',' ',text)
    text = re.sub(r'&[^\s]+', ' ', text)


    return text

In [ ]:
clean_data(dataset1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(dataset1)

                                                Tweets
0               miniso has these and theyre so cute   
1    convomfs original source vanilla raspberry zwi...
2    miniso turned into a gym and he was doing weig...
3    olixyt yes but i was too shy too ask  its okay...
4    miniso corporation has filed for fcc id approv...
..                                                 ...
195  rt sugarsanriio hello kitty cooler bag from mi...
196  rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
197  rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
198     wana____ miniso taught me shege thats why lool
199  rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...

[200 rows x 1 columns]


## Stopwords Removal

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
print(dataset1['Tweets']); 

import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
 
dataset1['Tweets'] = dataset1['Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(dataset1['Tweets'])

0                 miniso has these and theyre so cute   
1      convomfs original source vanilla raspberry zwi...
2      miniso turned into a gym and he was doing weig...
3      olixyt yes but i was too shy too ask  its okay...
4      miniso corporation has filed for fcc id approv...
                             ...                        
195    rt sugarsanriio hello kitty cooler bag from mi...
196    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
197    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
198       wana____ miniso taught me shege thats why lool
199    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
Name: Tweets, Length: 200, dtype: object
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0                                     miniso theyre cute
1      convomfs original source vanilla raspberry 

## Tokenizing

In [ ]:
dataset1['Tweets'] = dataset1['Tweets'].apply(word_tokenize)

print(dataset1['Tweets'])

0                                 [miniso, theyre, cute]
1      [convomfs, original, source, vanilla, raspberr...
2                [miniso, turned, gym, weights, infront]
3              [olixyt, yes, shy, ask, okay, appreciate]
4      [miniso, corporation, filed, fcc, id, approval...
                             ...                        
195    [rt, sugarsanriio, hello, kitty, cooler, bag, ...
196    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
197    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
198       [wana____, miniso, taught, shege, thats, lool]
199    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
Name: Tweets, Length: 200, dtype: object


In [ ]:
print(dataset1)

                                                Tweets
0                               [miniso, theyre, cute]
1    [convomfs, original, source, vanilla, raspberr...
2              [miniso, turned, gym, weights, infront]
3            [olixyt, yes, shy, ask, okay, appreciate]
4    [miniso, corporation, filed, fcc, id, approval...
..                                                 ...
195  [rt, sugarsanriio, hello, kitty, cooler, bag, ...
196  [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
197  [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
198     [wana____, miniso, taught, shege, thats, lool]
199  [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...

[200 rows x 1 columns]


## Hasil Akhir

In [ ]:
dataset1.to_csv('/content/gdrive/My Drive/PDB/preprocessing_PDB.csv', index = False)

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/PDB/preprocessing_PDB.csv')

In [ ]:
from textblob import TextBlob

def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity

def getPolarity(review):
    return TextBlob(review).sentiment.polarity

def analyze(score):
    if score  < 0:
        return '-1'
    elif score == 0:
        return '0'
    else:
        return '1'

In [ ]:
final_data = pd.DataFrame(data[['Tweets']])
final_data.head()

,Tweets
0,"['miniso', 'theyre', 'cute']"
1,"['convomfs', 'original', 'source', 'vanilla', ..."
2,"['miniso', 'turned', 'gym', 'weights', 'infront']"
3,"['olixyt', 'yes', 'shy', 'ask', 'okay', 'appre..."
4,"['miniso', 'corporation', 'filed', 'fcc', 'id'..."


In [ ]:
final_data['Subjectivity'] = final_data['Tweets'].apply(getSubjectivity)
final_data['Polarity'] =  final_data['Tweets'].apply(getPolarity)
final_data['TextBlob'] = final_data['Polarity'].apply(analyze)
final_data.head()

,Tweets,Subjectivity,Polarity,TextBlob
0,"['miniso', 'theyre', 'cute']",1.000000,0.500000,1
1,"['convomfs', 'original', 'source', 'vanilla', ...",0.394444,0.069444,1
2,"['miniso', 'turned', 'gym', 'weights', 'infront']",0.000000,0.000000,0
3,"['olixyt', 'yes', 'shy', 'ask', 'okay', 'appre...",0.500000,0.000000,0
4,"['miniso', 'corporation', 'filed', 'fcc', 'id'...",0.000000,0.000000,0


In [ ]:
tb_counts = final_data.TextBlob.value_counts()
tb_counts

0     108
1      81
-1     11
Name: TextBlob, dtype: int64

In [ ]:
final_data.to_csv('/content/gdrive/My Drive/PDB/Final_Data.csv', index = False)

In [ ]:
# Naive Bayes Classification

# Importing the libraries
import os
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from imblearn.over_sampling import SMOTE


In [ ]:
# Importing the dataset
df = pd.read_csv('/content/gdrive/My Drive/PDB/Final_Data.csv')

In [ ]:
#looking at the first 5 values of the dataset
df.head()

,Tweets,Subjectivity,Polarity,TextBlob
0,"['miniso', 'theyre', 'cute']",1.000000,0.500000,1
1,"['convomfs', 'original', 'source', 'vanilla', ...",0.394444,0.069444,1
2,"['miniso', 'turned', 'gym', 'weights', 'infront']",0.000000,0.000000,0
3,"['olixyt', 'yes', 'shy', 'ask', 'okay', 'appre...",0.500000,0.000000,0
4,"['miniso', 'corporation', 'filed', 'fcc', 'id'...",0.000000,0.000000,0


In [ ]:
df['TextBlob'].value_counts()

 0    108
 1     81
-1     11
Name: TextBlob, dtype: int64

In [ ]:
X = df['Tweets']
Y = df['TextBlob']
from sklearn.model_selection import train_test_split as tt
X_train, X_test, Y_train, Y_test = tt(X, Y,test_size=0.2, random_state=100)

In [ ]:
X_train.shape

(160,)

In [ ]:
X_test.shape

(40,)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(stop_words ='english')
vector.fit(X_train)

CountVectorizer(stop_words='english')

In [ ]:
vector.vocabulary_

{'_khunby': 0,
 'accepted': 1,
 'actu': 2,
 'actual': 3,
 'adewuyifm': 4,
 'ago': 5,
 'ahits': 6,
 'allabout_atz': 7,
 'amazon': 8,
 'amazonmallpk': 9,
 'amazonoutletmall': 10,
 'amber': 11,
 'amp': 12,
 'anxiety': 13,
 'anymore': 14,
 'appreciate': 15,
 'approval': 16,
 'arcade': 17,
 'arramiiidee': 18,
 'ask': 19,
 'askrl': 20,
 'ateez': 21,
 'attack': 22,
 'au': 23,
 'available': 24,
 'awake': 25,
 'backpack': 26,
 'bag': 27,
 'bags': 28,
 'bank': 29,
 'bare': 30,
 'baridura': 31,
 'basic': 32,
 'bcs': 33,
 'bea': 34,
 'bears': 35,
 'beautiful': 36,
 'berate': 37,
 'best': 38,
 'better': 39,
 'bev': 40,
 'bias': 41,
 'bisa': 42,
 'black': 43,
 'blenco': 44,
 'bloomrooms': 45,
 'body': 46,
 'boo': 47,
 'bottle': 48,
 'bought': 49,
 'box': 50,
 'boy': 51,
 'branches': 52,
 'brands': 53,
 'breaking': 54,
 'bro': 55,
 'btppubliceye': 56,
 'bulletin': 57,
 'bumblebee': 58,
 'bump': 59,
 'buy': 60,
 'carried': 61,
 'case': 62,
 'cashier': 63,
 'center': 64,
 'cfc_saada': 65,
 'cheap': 66,

In [ ]:
vector.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['_khunby',
 'accepted',
 'actu',
 'actual',
 'adewuyifm',
 'ago',
 'ahits',
 'allabout_atz',
 'amazon',
 'amazonmallpk',
 'amazonoutletmall',
 'amber',
 'amp',
 'anxiety',
 'anymore',
 'appreciate',
 'approval',
 'arcade',
 'arramiiidee',
 'ask',
 'askrl',
 'ateez',
 'attack',
 'au',
 'available',
 'awake',
 'backpack',
 'bag',
 'bags',
 'bank',
 'bare',
 'baridura',
 'basic',
 'bcs',
 'bea',
 'bears',
 'beautiful',
 'berate',
 'best',
 'better',
 'bev',
 'bias',
 'bisa',
 'black',
 'blenco',
 'bloomrooms',
 'body',
 'boo',
 'bottle',
 'bought',
 'box',
 'boy',
 'branches',
 'brands',
 'breaking',
 'bro',
 'btppubliceye',
 'bulletin',
 'bumblebee',
 'bump',
 'buy',
 'carried',
 'case',
 'cashier',
 'center',
 'cfc_saada',
 'cheap',
 'checkout',
 'cheolsoo',
 'choose',
 'cigarjjk',
 'closed',
 'cmb',
 'collection',
 'color',
 'coloured',
 'coming',
 'convomfs',
 'cool',
 'coolboy_joel',
 'cooler',
 'corgi',
 'corporation',
 'cosmic',
 'couple',
 'cowboy',
 'crossbody',
 'cup',
 'cute',

In [ ]:
X_train_transformed =vector.transform(X_train)
X_test_transformed =vector.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_transformed,Y_train)
y_pred = model.predict(X_test_transformed)
y_pred_prob = model.predict_proba(X_test_transformed)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
print(confusion_matrix(Y_test,y_pred))
print()
print(accuracy_score(Y_test,y_pred))

[[ 0  4  1]
 [ 0 23  1]
 [ 0  1 10]]

0.825


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         5
           0       0.82      0.96      0.88        24
           1       0.83      0.91      0.87        11

    accuracy                           0.82        40
   macro avg       0.55      0.62      0.58        40
weighted avg       0.72      0.82      0.77        40



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
